In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## import data from excel

In [2]:
df = pd.read_excel('filtered_cars.xlsx')

## make Car Brands with Unique Car brand ID

In [3]:
# Extract unique car brands and assign a unique ID to each
car_brands = df[['make']].drop_duplicates().reset_index(drop=True)
#Fist 3 letters of make + index + 100 = Car_id
car_brands['brand_id'] = car_brands['make'].str[:3].str.upper() + (car_brands.index + 101).astype(str)

# Display the resulting table
print(car_brands)

            make brand_id
0        Hyundai   HYU101
1            Bmw   BMW102
2         Toyota   TOY103
3       Mahindra   MAH104
4  Mercedes-Benz   MER105
5           Tata   TAT106
6          Honda   HON107
7           Ford   FOR108
8  Maruti Suzuki   MAR109


In [4]:
print(df['make'].value_counts())

make
Maruti Suzuki    79
Hyundai          75
Mahindra         59
Tata             55
Toyota           45
Mercedes-Benz    39
Honda            36
Ford             27
Bmw              21
Name: count, dtype: int64


Add the BrandID to the df

In [5]:
brand_id_map = dict(zip(car_brands['make'], car_brands['brand_id']))
# Add a Brand_ID column to the df DataFrame
df['Brand_ID'] = df['make'].map(brand_id_map)
# Display the updated DataFrame
df.head()

,make,model,variant,Price_INR,engine_cc,Cylinders,fuel_capacity,Fuel_type,Body_type,mileage,...,Seating_capacity,Transmission_type,warranty,Sun_visor,Ventilation_system,ABS,Airbags,color,rating,Brand_ID
0,Hyundai,Venue,1.0 Turbo Gdi Mt Sx Dual Tone,974000,998,4,45,PETROL,SUV,18.27,...,5,Manual,2,True,True,True,True,Silver,5,HYU101
1,Bmw,X3,Xdrive20D Xline,5600000,1995,4,67,DIESEL,SUV,18.56,...,5,Automatic,2,True,True,True,True,Green,5,BMW102
2,Toyota,Fortuner,Trd Celebratory Edition,3420000,2755,4,80,DIESEL,SUV,12.90,...,7,Automatic,3,True,True,True,True,Silver,5,TOY103
3,Hyundai,Creta,1.6 Vtvt Sx,1232534,1591,4,55,PETROL,SUV,15.29,...,5,Manual,3,True,True,True,True,Maroon,5,HYU101
4,Mahindra,Nuvosport,N6 Amt,972359,1493,3,60,DIESEL,SUV,16.21,...,7,Automatic,2,True,True,True,True,Yellow,4,MAH104


 ## make the dealers_df
The `dealers_df` DataFrame contains information about car dealers, including their personal details, dealership information, and unique usernames. Each dealer is associated with a car brand, and the brand is mapped to a unique `brand_id` using the `car_brands` DataFrame. The columns in `dealers_df` include:

- Name: Dealer's full name
- password: Password for dealer login (for demonstration purposes)
- Email: Dealer's email address
- Address: Dealer's address
- City: City where the dealer is located
- pin: Postal code of the dealer's location
- Car_Company: The car brand the dealer represents
- Age: Dealer's age
- gender: Dealer's gender
- userName: A unique username generated for each dealer, combining parts of their name and brand_id

This DataFrame is used to create the dealers table in the MySQL database and to manage dealer-related operations in the workflow.

In [6]:
import random
import string

def generate_dealer_usernames(df):
    """
    Generate unique dealer usernames such that if two dealers have the same alphabetic part (e.g., D_ARUNTAT),
    the first gets 001, the second gets 003, the third 005, etc. (odd numbers only).
    """
    # Create the base username (without the number)
    base_usernames = (
        "D_" +
        df['Name'].apply(lambda x: x.split()[0].upper()) +
        df['brand_id'].str[:3].str.upper()
    )
    # Count occurrences for each base username
    username_counts = {}
    usernames = []
    for base in base_usernames:
        count = username_counts.get(base, 0)
        # Odd numbers: 001, 002, 003, ...
        num = 1 + count
        usernames.append(f"{base}{str(num).zfill(3)}")
        username_counts[base] = count + 1
    return usernames

# Create a DataFrame for dealers with realistic details
def generate_password(name):
    """Generate password as name + '@' + 3 random digits + '$'."""
    name = name.split()[0]
    digits = ''.join(random.choices(string.digits, k=3))
    return f"{name.replace(' ', '')}@{digits}$"

dealers_data = {
    'Name': [
        'Arun Kumar', 'Priya Sharma', 'Ramesh Babu', 'Divya Menon', 'Karthik Raja', 'Sneha Iyer',
        'Vijay Anand', 'Meera Nair', 'Suresh Reddy', 'Arun Sharma', 'Rajesh Khanna', 'Pooja Patel',
        'Senthil Kumar', 'Lakshmi Narayanan', 'Sivakumar Subramanian', 'Revathi Balaji', 'Gopinath Ram',
        'Shanthi Ramesh', 'Balaji Srinivasan', 'Kavitha Murugan', 'Deepak Joshi', 'Anjali Verma',
        'Manoj Pillai', 'Ritu Agarwal', 'Harish Kumar', 'Neha Suresh', 'Prakash Menon', 'Asha Rani',
        'Sanjay Gupta', 'Vidya Krishnan'
    ],
    'password': [generate_password(name) for name in [
        'Arun Kumar', 'Priya Sharma', 'Ramesh Babu', 'Divya Menon', 'Karthik Raja', 'Sneha Iyer',
        'Vijay Anand', 'Meera Nair', 'Suresh Reddy', 'Arun Sharma', 'Rajesh Khanna', 'Pooja Patel',
        'Senthil Kumar', 'Lakshmi Narayanan', 'Sivakumar Subramanian', 'Revathi Balaji', 'Gopinath Ram',
        'Shanthi Ramesh', 'Balaji Srinivasan', 'Kavitha Murugan', 'Deepak Joshi', 'Anjali Verma',
        'Manoj Pillai', 'Ritu Agarwal', 'Harish Kumar', 'Neha Suresh', 'Prakash Menon', 'Asha Rani',
        'Sanjay Gupta', 'Vidya Krishnan'
    ]],
    'Email': [
        'arun.kumar@gmail.com', 'priya.sharma@gmail.com', 'ramesh.babu@gmail.com',
        'divya.menon@gmail.com', 'karthik.raja@gmail.com', 'sneha.iyer@gmail.com',
        'vijay.anand@gmail.com', 'meera.nair@gmail.com', 'suresh.reddy@gmail.com',
        'arun.gupta@gmail.com', 'rajesh.khanna@gmail.com', 'pooja.patel@gmail.com',
        'senthil.kumar@tnmail.com', 'lakshmi.narayanan@tnmail.com', 'sivakumar.subramanian@tnmail.com',
        'revathi.balaji@tnmail.com', 'gopinath.ram@tnmail.com', 'shanthi.ramesh@tnmail.com',
        'balaji.srinivasan@tnmail.com', 'kavitha.murugan@tnmail.com', 'deepak.joshi@gmail.com',
        'anjali.verma@gmail.com', 'manoj.pillai@gmail.com', 'ritu.agarwal@gmail.com',
        'harish.kumar@gmail.com', 'neha.suresh@gmail.com', 'prakash.menon@gmail.com',
        'asha.rani@gmail.com', 'sanjay.gupta@gmail.com', 'vidya.krishnan@gmail.com'
    ],
    'Phone': [
        '9876543210', '9123456780', '9988776655', '9876501234', '9123409876', '9988123456',
        '9876123450', '9123987654', '9988345678', '9876234567', '9123567890', '9988456123',
        '9876345678', '9123678901', '9988561234', '9876456789', '9123789012', '9988672345',
        '9876567890', '9123890123', '9876612345', '9123912345', '9988712345', '9876623456',
        '9123923456', '9988723456', '9876634567', '9123934567', '9988734567', '9876645678'
    ],
    'Address': [
        'No. 12, Anna Nagar, Chennai', 'No. 45, Gandhi Road, Coimbatore', 'No. 78, MG Street, Madurai',
        'No. 23, Park Avenue, Tiruchirappalli', 'No. 56, Raja Street, Salem', 'No. 89, Lake View, Erode',
        'No. 34, Temple Road, Tirunelveli', 'No. 67, Green Park, Vellore', 'No. 90, Sunrise Avenue, Thoothukudi',
        'No. 11, Hilltop Road, Dindigul', 'No. 22, River Side, Thanjavur', 'No. 33, Ocean Drive, Kanchipuram',
        'No. 101, Gandhi Salai, Karur', 'No. 202, Mela Street, Namakkal', 'No. 303, Bazaar Road, Theni',
        'No. 404, College Road, Sivakasi', 'No. 505, Main Road, Cuddalore', 'No. 606, Market Street, Tiruppur',
        'No. 707, South Street, Hosur', 'No. 808, North Street, Nagercoil', 'No. 909, West End, Chennai',
        'No. 123, Lotus Colony, Madurai', 'No. 234, Rose Street, Salem', 'No. 345, Jasmine Road, Erode',
        'No. 456, Orchid Lane, Vellore', 'No. 567, Lily Avenue, Thanjavur', 'No. 678, Tulip Street, Karur',
        'No. 789, Daisy Road, Namakkal', 'No. 890, Sunflower Lane, Theni', 'No. 901, Marigold Road, Sivakasi'
    ],
    'City': [
        'Chennai', 'Coimbatore', 'Madurai', 'Tiruchirappalli', 'Salem', 'Erode',
        'Tirunelveli', 'Vellore', 'Thoothukudi', 'Dindigul', 'Thanjavur', 'Kanchipuram',
        'Karur', 'Namakkal', 'Theni', 'Sivakasi', 'Cuddalore', 'Tiruppur', 'Hosur', 'Nagercoil',
        'Chennai', 'Madurai', 'Salem', 'Erode', 'Vellore', 'Thanjavur', 'Karur', 'Namakkal', 'Theni', 'Sivakasi'
    ],
    'pincode': [
        600001, 641001, 625001, 620001, 636001, 638001, 627001, 632001, 628001, 624001, 613001, 631501,
        639001, 637001, 625531, 626123, 607001, 641601, 635109, 629001, 600002, 625002, 636002, 638002,
        632002, 613002, 639002, 637002, 625532, 626124
    ],
    'Car_Company': [
        'Tata', 'Hyundai', 'Maruti Suzuki', 'Toyota', 'Ford', 'Mahindra',
        'Honda', 'Mercedes-Benz', 'Bmw', 'Tata', 'Hyundai', 'Maruti Suzuki',
        'Toyota', 'Ford', 'Mahindra', 'Honda', 'Mercedes-Benz', 'Bmw', 'Tata', 'Hyundai',
        'Maruti Suzuki', 'Toyota', 'Ford', 'Mahindra', 'Honda', 'Mercedes-Benz', 'Bmw', 'Tata', 'Hyundai', 'Maruti Suzuki'
    ],
    'Age': [
        35, 40, 28, 45, 32, 38, 29, 50, 33, 41, 36, 30, 39, 42, 37, 34, 48, 31, 44, 29,
        38, 27, 41, 36, 33, 39, 35, 32, 30, 28
    ],
    'gender': [
        'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE',
        'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE',
        'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE'
    ],
    'description': [
        "Experienced Tata dealer in Chennai.\nCommitted to customer satisfaction.\nExpert in new and used cars.\nOffers a wide range of Tata vehicles, from hatchbacks to SUVs.\nProvides personalized recommendations based on customer needs.\nKnown for transparent pricing and after-sales support.",
        "Leading Hyundai dealer in Coimbatore.\nKnown for transparent deals.\nPassionate about automobiles.\nSpecializes in the latest Hyundai models and technology.\nAssists customers with finance and insurance options.\nFocuses on building long-term relationships with clients.",
        "Maruti Suzuki specialist in Madurai.\nFriendly and knowledgeable staff.\nTrusted by local families.\nProvides test drives and detailed car comparisons.\nOffers attractive exchange and loyalty bonuses.\nEnsures a hassle-free buying experience for all customers.",
        "Toyota dealership in Tiruchirappalli.\nFocus on quality service.\nWide range of models available.\nExpert in both personal and commercial Toyota vehicles.\nGuides customers through every step of the purchase process.\nMaintains a strong reputation for reliability and trust.",
        "Ford dealer serving Salem region.\nCustomer-first approach.\nOffers best exchange deals.\nProvides maintenance packages and extended warranties.\nHelps customers choose the right Ford for their lifestyle.\nActive in local community events and sponsorships.",
        "Mahindra expert in Erode.\nDedicated to after-sales support.\nModern showroom facilities.\nSpecializes in Mahindra SUVs and utility vehicles.\nOffers quick service turnaround and genuine parts.\nKnown for customer loyalty and repeat business.",
        "Honda dealership in Tirunelveli.\nPersonalized buying experience.\nStrong local reputation.\nOffers the full range of Honda cars and accessories.\nProvides expert advice on fuel efficiency and maintenance.\nHosts regular customer engagement programs.",
        "Mercedes-Benz dealer in Vellore.\nLuxury car specialist.\nPremium customer care.\nShowcases the latest Mercedes-Benz innovations.\nOffers exclusive test drive experiences.\nProvides tailored finance solutions for luxury buyers.",
        "BMW dealership in Thoothukudi.\nLatest models on display.\nFinance and insurance assistance.\nFocuses on BMW performance and safety features.\nOrganizes BMW owner events and workshops.\nEnsures a premium experience from inquiry to delivery.",
        "Tata dealer in Dindigul.\nQuick delivery and service.\nFamily-owned business.\nOffers special deals for first-time buyers.\nProvides detailed vehicle demonstrations.\nActive in supporting local causes and charities.",
        "Hyundai expert in Thanjavur.\nAward-winning sales team.\nFocus on customer loyalty.\nKeeps up with the latest Hyundai launches.\nProvides comprehensive after-sales care.\nKnown for fast and efficient service appointments.",
        "Maruti Suzuki dealer in Kanchipuram.\nAffordable pricing.\nExcellent service record.\nOffers a variety of Maruti Suzuki models.\nGuides customers on best value options.\nMaintains high customer satisfaction ratings.",
        "Toyota dealership in Karur.\nWide selection of vehicles.\nKnown for honest advice.\nAssists with trade-ins and upgrades.\nProvides detailed information on Toyota safety features.\nHosts regular service camps for customers.",
        "Ford dealer in Namakkal.\nFlexible financing options.\nStrong community presence.\nOffers Ford certified pre-owned vehicles.\nProvides transparent billing and documentation.\nEngages with customers through feedback and surveys.",
        "Mahindra expert in Theni.\nSpecializes in SUVs.\nCustomer-centric approach.\nOffers test drives on all Mahindra models.\nProvides detailed comparisons with competitors.\nFocuses on building trust and long-term relationships.",
        "Honda dealership in Sivakasi.\nFriendly staff and quick service.\nTrusted by repeat buyers.\nOffers Honda genuine accessories and parts.\nProvides maintenance reminders and tips.\nOrganizes customer appreciation events.",
        "Mercedes-Benz dealer in Cuddalore.\nExclusive luxury experience.\nTailored solutions for clients.\nShowcases AMG and special edition models.\nOffers personalized delivery ceremonies.\nProvides 24/7 customer support for owners.",
        "BMW dealership in Tiruppur.\nCutting-edge technology.\nComprehensive after-sales support.\nFocuses on BMW M series and electric vehicles.\nProvides expert guidance on customization options.\nMaintains a dedicated service team for premium care.",
        "Tata dealer in Hosur.\nFocus on value for money.\nEfficient service department.\nOffers Tata fleet solutions for businesses.\nProvides extended warranty and roadside assistance.\nKnown for transparent and honest communication.",
        "Hyundai expert in Nagercoil.\nModern showroom.\nCustomer satisfaction guaranteed.\nOffers Hyundai digital buying experience.\nProvides quick loan approvals and paperwork.\nHosts regular car care workshops for customers.",
        "Maruti Suzuki dealer in Chennai.\nBest deals on new cars.\nStrong referral network.\nOffers Maruti Suzuki subscription and leasing.\nProvides detailed cost of ownership analysis.\nActive in local auto expos and events.",
        "Toyota dealership in Madurai.\nExperienced sales consultants.\nWide range of accessories.\nOffers Toyota certified used cars.\nProvides insurance and registration assistance.\nFocuses on customer education and transparency.",
        "Ford dealer in Salem.\nTransparent pricing policy.\nQuick turnaround on deliveries.\nOffers Ford service packages and AMC.\nProvides detailed vehicle walkarounds.\nKnown for prompt response to customer queries.",
        "Mahindra expert in Erode.\nKnown for reliability.\nExcellent trade-in offers.\nSpecializes in Mahindra commercial vehicles.\nProvides on-site vehicle demonstrations.\nMaintains a strong network of satisfied customers.",
        "Honda dealership in Vellore.\nPersonalized test drives.\nFlexible payment plans.\nOffers Honda connect and digital services.\nProvides regular service reminders.\nFocuses on customer convenience and comfort.",
        "Mercedes-Benz dealer in Thanjavur.\nLuxury redefined.\nAttention to every detail.\nShowcases the latest Mercedes-Benz models.\nOffers exclusive loyalty programs.\nProvides dedicated relationship managers.",
        "BMW dealership in Karur.\nPremium selection.\nCustomer-focused team.\nOffers BMW certified used cars.\nProvides expert advice on BMW technology.\nHosts exclusive BMW owner events.",
        "Tata dealer in Namakkal.\nAffordable maintenance.\nFriendly neighborhood dealer.\nOffers Tata service on wheels.\nProvides quick turnaround for repairs.\nKnown for community involvement.",
        "Hyundai expert in Theni.\nLatest models available.\nPrompt customer support.\nOffers Hyundai blue link connected services.\nProvides detailed product brochures.\nFocuses on customer retention.",
        "Maruti Suzuki dealer in Sivakasi.\nTrusted by generations.\nBest-in-class service.\nOffers Maruti Suzuki driving school.\nProvides quick insurance claim processing.\nMaintains a high repeat customer rate."
    ]
}

# Create the DataFrame
dealers_df = pd.DataFrame(dealers_data)

# Map Car_Company to brand_id using car_brands DataFrame
brand_id_map = dict(zip(car_brands['make'], car_brands['brand_id']))
dealers_df['brand_id'] = dealers_df['Car_Company'].map(brand_id_map)


# Generate userName using the custom function
dealers_df['userName'] = generate_dealer_usernames(dealers_df)


dealers_df.drop(['brand_id'], axis=1, inplace=True)
# Display the dealers DataFrame
dealers_df

,Name,password,Email,Phone,Address,City,pincode,Car_Company,Age,gender,description,userName
0,Arun Kumar,Arun@506$,arun.kumar@gmail.com,9876543210,"No. 12, Anna Nagar, Chennai",Chennai,600001,Tata,35,MALE,Experienced Tata dealer in Chennai.\nCommitted...,D_ARUNTAT001
1,Priya Sharma,Priya@028$,priya.sharma@gmail.com,9123456780,"No. 45, Gandhi Road, Coimbatore",Coimbatore,641001,Hyundai,40,FEMALE,Leading Hyundai dealer in Coimbatore.\nKnown f...,D_PRIYAHYU001
2,Ramesh Babu,Ramesh@541$,ramesh.babu@gmail.com,9988776655,"No. 78, MG Street, Madurai",Madurai,625001,Maruti Suzuki,28,MALE,Maruti Suzuki specialist in Madurai.\nFriendly...,D_RAMESHMAR001
3,Divya Menon,Divya@564$,divya.menon@gmail.com,9876501234,"No. 23, Park Avenue, Tiruchirappalli",Tiruchirappalli,620001,Toyota,45,FEMALE,Toyota dealership in Tiruchirappalli.\nFocus o...,D_DIVYATOY001
4,Karthik Raja,Karthik@097$,karthik.raja@gmail.com,9123409876,"No. 56, Raja Street, Salem",Salem,636001,Ford,32,MALE,Ford dealer serving Salem region.\nCustomer-fi...,D_KARTHIKFOR001
5,Sneha Iyer,Sneha@010$,sneha.iyer@gmail.com,9988123456,"No. 89, Lake View, Erode",Erode,638001,Mahindra,38,FEMALE,Mahindra expert in Erode.\nDedicated to after-...,D_SNEHAMAH001
6,Vijay Anand,Vijay@519$,vijay.anand@gmail.com,9876123450,"No. 34, Temple Road, Tirunelveli",Tirunelveli,627001,Honda,29,MALE,Honda dealership in Tirunelveli.\nPersonalized...,D_VIJAYHON001
7,Meera Nair,Meera@826$,meera.nair@gmail.com,9123987654,"No. 67, Green Park, Vellore",Vellore,632001,Mercedes-Benz,50,FEMALE,Mercedes-Benz dealer in Vellore.\nLuxury car s...,D_MEERAMER001
8,Suresh Reddy,Suresh@866$,suresh.reddy@gmail.com,9988345678,"No. 90, Sunrise Avenue, Thoothukudi",Thoothukudi,628001,Bmw,33,MALE,BMW dealership in Thoothukudi.\nLatest models ...,D_SURESHBMW001
9,Arun Sharma,Arun@955$,arun.gupta@gmail.com,9876234567,"No. 11, Hilltop Road, Dindigul",Dindigul,624001,Tata,41,FEMALE,Tata dealer in Dindigul.\nQuick delivery and s...,D_ARUNTAT002


In [7]:
# Map each brand to its available dealer usernames
brand_to_dealers = dealers_df.groupby('Car_Company')['userName'].apply(list).to_dict()

# Function to assign a random dealer username for each car based on its Brand_ID
def assign_dealer_username(row):
    # Find the brand name for this car
    brand = row['make']
    # Get the list of dealer usernames for this brand
    dealer_usernames = brand_to_dealers.get(brand, [])
    if dealer_usernames:
        return np.random.choice(dealer_usernames)
    else:
        return np.nan  # No dealer for this brand

# Assign DealerUsername to each car in df
df['DealerUsername'] = df.apply(assign_dealer_username, axis=1)

# Check that the assigned dealer's brand matches the car's brand
def check_brand_match(row):
    if pd.isna(row['DealerUsername']):
        return False
    dealer_row = dealers_df[dealers_df['userName'] == row['DealerUsername']]
    if dealer_row.empty:
        return False
    return dealer_row.iloc[0]['Car_Company'] == row['make']

df['DealerBrandMatch'] = df.apply(check_brand_match, axis=1)

# Display a summary of the check
print("All DealerUsername assignments have matching brands:",
      df['DealerBrandMatch'].all())

# Drop the check column after verification
df.drop(columns=['DealerBrandMatch'], inplace=True)

# Show the updated DataFrame
df[['make', 'model', 'Brand_ID', 'DealerUsername']].head()

All DealerUsername assignments have matching brands: True


,make,model,Brand_ID,DealerUsername
0,Hyundai,Venue,HYU101,D_PRIYAHYU001
1,Bmw,X3,BMW102,D_SURESHBMW001
2,Toyota,Fortuner,TOY103,D_DIVYATOY001
3,Hyundai,Creta,HYU101,D_PRIYAHYU001
4,Mahindra,Nuvosport,MAH104,D_SNEHAMAH001


In [8]:
# Replace Car_Company in dealers_df with the corresponding brand_id from car_brands
dealers_df['Brand_ID'] = dealers_df['Car_Company'].map(brand_id_map)
dealers_df.head()

,Name,password,Email,Phone,Address,City,pincode,Car_Company,Age,gender,description,userName,Brand_ID
0,Arun Kumar,Arun@506$,arun.kumar@gmail.com,9876543210,"No. 12, Anna Nagar, Chennai",Chennai,600001,Tata,35,MALE,Experienced Tata dealer in Chennai.\nCommitted...,D_ARUNTAT001,TAT106
1,Priya Sharma,Priya@028$,priya.sharma@gmail.com,9123456780,"No. 45, Gandhi Road, Coimbatore",Coimbatore,641001,Hyundai,40,FEMALE,Leading Hyundai dealer in Coimbatore.\nKnown f...,D_PRIYAHYU001,HYU101
2,Ramesh Babu,Ramesh@541$,ramesh.babu@gmail.com,9988776655,"No. 78, MG Street, Madurai",Madurai,625001,Maruti Suzuki,28,MALE,Maruti Suzuki specialist in Madurai.\nFriendly...,D_RAMESHMAR001,MAR109
3,Divya Menon,Divya@564$,divya.menon@gmail.com,9876501234,"No. 23, Park Avenue, Tiruchirappalli",Tiruchirappalli,620001,Toyota,45,FEMALE,Toyota dealership in Tiruchirappalli.\nFocus o...,D_DIVYATOY001,TOY103
4,Karthik Raja,Karthik@097$,karthik.raja@gmail.com,9123409876,"No. 56, Raja Street, Salem",Salem,636001,Ford,32,MALE,Ford dealer serving Salem region.\nCustomer-fi...,D_KARTHIKFOR001,FOR108


In [9]:
# Drop the 'Car_Company' column from dealers_df
dealers_df.drop(columns=['Car_Company'], inplace=True)
dealers_df.head()

,Name,password,Email,Phone,Address,City,pincode,Age,gender,description,userName,Brand_ID
0,Arun Kumar,Arun@506$,arun.kumar@gmail.com,9876543210,"No. 12, Anna Nagar, Chennai",Chennai,600001,35,MALE,Experienced Tata dealer in Chennai.\nCommitted...,D_ARUNTAT001,TAT106
1,Priya Sharma,Priya@028$,priya.sharma@gmail.com,9123456780,"No. 45, Gandhi Road, Coimbatore",Coimbatore,641001,40,FEMALE,Leading Hyundai dealer in Coimbatore.\nKnown f...,D_PRIYAHYU001,HYU101
2,Ramesh Babu,Ramesh@541$,ramesh.babu@gmail.com,9988776655,"No. 78, MG Street, Madurai",Madurai,625001,28,MALE,Maruti Suzuki specialist in Madurai.\nFriendly...,D_RAMESHMAR001,MAR109
3,Divya Menon,Divya@564$,divya.menon@gmail.com,9876501234,"No. 23, Park Avenue, Tiruchirappalli",Tiruchirappalli,620001,45,FEMALE,Toyota dealership in Tiruchirappalli.\nFocus o...,D_DIVYATOY001,TOY103
4,Karthik Raja,Karthik@097$,karthik.raja@gmail.com,9123409876,"No. 56, Raja Street, Salem",Salem,636001,32,MALE,Ford dealer serving Salem region.\nCustomer-fi...,D_KARTHIKFOR001,FOR108


### drop the make column in the df

In [10]:
df.drop(['make'], axis=1, inplace=True)
df.head()

,model,variant,Price_INR,engine_cc,Cylinders,fuel_capacity,Fuel_type,Body_type,mileage,Gears,...,Transmission_type,warranty,Sun_visor,Ventilation_system,ABS,Airbags,color,rating,Brand_ID,DealerUsername
0,Venue,1.0 Turbo Gdi Mt Sx Dual Tone,974000,998,4,45,PETROL,SUV,18.27,6,...,Manual,2,True,True,True,True,Silver,5,HYU101,D_PRIYAHYU001
1,X3,Xdrive20D Xline,5600000,1995,4,67,DIESEL,SUV,18.56,8,...,Automatic,2,True,True,True,True,Green,5,BMW102,D_SURESHBMW001
2,Fortuner,Trd Celebratory Edition,3420000,2755,4,80,DIESEL,SUV,12.90,6,...,Automatic,3,True,True,True,True,Silver,5,TOY103,D_DIVYATOY001
3,Creta,1.6 Vtvt Sx,1232534,1591,4,55,PETROL,SUV,15.29,6,...,Manual,3,True,True,True,True,Maroon,5,HYU101,D_PRIYAHYU001
4,Nuvosport,N6 Amt,972359,1493,3,60,DIESEL,SUV,16.21,5,...,Automatic,2,True,True,True,True,Yellow,4,MAH104,D_SNEHAMAH001


## Create a method which generate the Unique CarId from some car deatils

This method generates a unique CarId for each car by combining key attributes such as make, model, variant, price, fuel type, body type, colour, and type into a single string, then applying an MD5 hash to ensure uniqueness. The CarId typically consists of a short tag from the model name and a 6-character hash suffix, making it both human-readable and collision-resistant for database indexing and reference.

In [11]:
# Define Car Code generation function
import hashlib
import uuid

# def generate_car_code(row):
#     """
#     Generate a unique car code using various identifying attributes.
#     """
#     base_str = f"{row['model']}_{row['variant']}_{row['Price_INR']}_{row['Fuel_type']}_{row['Body_type']}_{row['Colour']}_{row['DealerUsername']}"
#     model = row['model']
#     # Short tag (first 3 letters of model)
#     short_tag = (model[:3].upper() if isinstance(model, str) else "XXX")

#     # Suffix hash: use first 6 hex characters from MD5 hash of all combined info
#     unique_hash = hashlib.md5(base_str.encode()).hexdigest()[:6].upper()
    
#     return f"{short_tag}{unique_hash}"


def generate_car_code():

# Generate a random UUID and convert it to a string
    random_id = str(uuid.uuid4())
    return random_id.upper()

In [12]:
generate_car_code()

'1F4B6FAF-7D2F-4470-90D3-31241A2A394D'

In [13]:
print(len(generate_car_code()))

36


add the CarId into the df

In [14]:
# Generate a unique Car_id for each car using the generate_car_code method
df['Car_id'] = df.apply(lambda row: generate_car_code(), axis=1)

In [15]:
df[['model', 'variant', 'Price_INR','Fuel_type','Body_type','color','DealerUsername','Car_id']].head()

,model,variant,Price_INR,Fuel_type,Body_type,color,DealerUsername,Car_id
0,Venue,1.0 Turbo Gdi Mt Sx Dual Tone,974000,PETROL,SUV,Silver,D_PRIYAHYU001,E7DDE5E8-3C28-44C7-81E3-0655C9AC903D
1,X3,Xdrive20D Xline,5600000,DIESEL,SUV,Green,D_SURESHBMW001,75DE4879-925F-430B-B03C-B34C51A0A539
2,Fortuner,Trd Celebratory Edition,3420000,DIESEL,SUV,Silver,D_DIVYATOY001,FC2A430C-764B-4891-822D-5D7367CF90B6
3,Creta,1.6 Vtvt Sx,1232534,PETROL,SUV,Maroon,D_PRIYAHYU001,5B7DD4BD-5D1F-48E8-895B-4DBA501349CD
4,Nuvosport,N6 Amt,972359,DIESEL,SUV,Yellow,D_SNEHAMAH001,02CC8F43-A3DA-48B8-AE4C-DE9AAFA8350F


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   model               436 non-null    object 
 1   variant             436 non-null    object 
 2   Price_INR           436 non-null    int64  
 3   engine_cc           436 non-null    int64  
 4   Cylinders           436 non-null    int64  
 5   fuel_capacity       436 non-null    int64  
 6   Fuel_type           436 non-null    object 
 7   Body_type           436 non-null    object 
 8   mileage             436 non-null    float64
 9   Gears               436 non-null    int64  
 10  Power_steering      436 non-null    object 
 11  Keyless_entry       436 non-null    bool   
 12  power_hp            436 non-null    float64
 13  torque_nm           436 non-null    float64
 14  Seating_capacity    436 non-null    int64  
 15  Transmission_type   436 non-null    object 
 16  warranty

Check that if there any duplicate CarId avalable or not

In [17]:
# Check for duplicate Car_id
duplicate_car_ids = df['Car_id'].duplicated().any()
print(f"Are there any duplicate Car_id values? {duplicate_car_ids}")

Are there any duplicate Car_id values? False


## Make Customers_df
The `customers_df` DataFrame contains information about individual customers, including their personal details and unique usernames. Each row represents a customer with the following columns:

- **Name**: Customer's full name  
- **password**: Password for customer login (for demonstration purposes)  
- **Email**: Customer's email address  
- **Address**: Customer's residential address  
- **City**: City where the customer resides  
- **pin**: Postal code of the customer's address  
- **gender**: Customer's gender  
- **userName**: A unique username generated for each customer, combining parts of their name and a sequence number to ensure uniqueness  

This DataFrame is used to create the customers table in the MySQL database and to manage customer-related operations in the workflow.

In [18]:
def generate_password(name):
    """Generate password as firstname + '@' + 3 random digits + '$'."""
    first_name = name.split()[0]
    digits = ''.join(random.choices(string.digits, k=3))
    return f"{first_name}@{digits}#"

# Generate unique usernames for all customers
def generate_customer_usernames(df):
    """
    Generate unique customer usernames in the format:
    C_<FIRSTNAME><CITY3><NNN>
    where NNN is a sequence number (001, 002, ...) for duplicates.
    """
    base_usernames = (
        "C_" +
        df['Name'].apply(lambda x: x.split()[0].upper()) +
        df['City'].str[:3].str.upper()
    )
    username_counts = {}
    usernames = []
    for base in base_usernames:
        count = username_counts.get(base, 0)
        num = count + 1
        usernames.append(f"{base}{str(num).zfill(3)}")
        username_counts[base] = num
    return usernames

# Create a DataFrame for 15 unique customers
customers_data = {
    'Name': [
        'Harish Pillai', 'Swetha Reddy', 'Swetha Shetty', 'Deepa Suresh', 'Rahul Nambiar',
        'Lakshmi Prasad', 'Sandeep Varma', 'Preeti Desai', 'Ajay Kulkarni', 'Rina Fernandes',
        'Vivek Joshi', 'Meena Chatterjee', 'Sahil Kapoor', 'Tanvi Bansal', 'Yash Mehra',
        'Anita Sharma', 'Rohit Verma', 'Kiran Rao', 'Sunita Menon', 'Amitabh Singh',
        'Priya Das', 'Nitin Patil', 'Sneha Agarwal', 'Arjun Nair', 'Divya Gupta'
    ],
    'password': [generate_password(name) for name in [
        'Harish Pillai', 'Swetha Reddy', 'Swetha Shetty', 'Deepa Suresh', 'Rahul Nambiar',
        'Lakshmi Prasad', 'Sandeep Varma', 'Preeti Desai', 'Ajay Kulkarni', 'Rina Fernandes',
        'Vivek Joshi', 'Meena Chatterjee', 'Sahil Kapoor', 'Tanvi Bansal', 'Yash Mehra',
        'Anita Sharma', 'Rohit Verma', 'Kiran Rao', 'Sunita Menon', 'Amitabh Singh',
        'Priya Das', 'Nitin Patil', 'Sneha Agarwal', 'Arjun Nair', 'Divya Gupta'
    ]],
    'Email': [
        'harish.pillai@gmail.com', 'swetha.reddy@gmail.com', 'manoj.shetty@gmail.com',
        'deepa.suresh@gmail.com', 'rahul.nambiar@gmail.com', 'lakshmi.prasad@gmail.com',
        'sandeep.varma@gmail.com', 'preeti.desai@gmail.com', 'ajay.kulkarni@gmail.com',
        'rina.fernandes@gmail.com', 'vivek.joshi@gmail.com', 'meena.chatterjee@gmail.com',
        'sahil.kapoor@gmail.com', 'tanvi.bansal@gmail.com', 'yash.mehra@gmail.com',
        'anita.sharma@gmail.com', 'rohit.verma@gmail.com', 'kiran.rao@gmail.com',
        'sunita.menon@gmail.com', 'amitabh.singh@gmail.com', 'priya.das@gmail.com',
        'nitin.patil@gmail.com', 'sneha.agarwal@gmail.com', 'arjun.nair@gmail.com',
        'divya.gupta@gmail.com'
    ],
    'Address': [
        'No. 10, Lotus Street', 'No. 21, Maple Avenue', 'No. 32, Palm Grove',
        'No. 43, Cedar Lane', 'No. 54, Jasmine Road', 'No. 65, Orchid Enclave',
        'No. 76, Pine Residency', 'No. 87, Willow Park', 'No. 98, Rose Drive',
        'No. 109, Tulip Block', 'No. 120, Banyan Street', 'No. 131, Peepal Lane',
        'No. 142, Ashoka Marg', 'No. 153, Neem Avenue', 'No. 164, Mango Street',
        'No. 175, Lotus Enclave', 'No. 186, Maple Residency', 'No. 197, Palm Avenue',
        'No. 208, Cedar Residency', 'No. 219, Jasmine Enclave', 'No. 230, Orchid Street',
        'No. 241, Pine Avenue', 'No. 252, Willow Residency', 'No. 263, Rose Enclave',
        'No. 274, Tulip Avenue'
    ],
    'Phone': [
        '9876543210', '9123456780', '9988776655', '9876501234', '9123409876',
        '9001122334', '9090909090', '9887766554', '9876123450', '9123987654',
        '9001234567', '9887654321', '9876098765', '9123567890', '9001987654',
        '9887012345', '9876987654', '9123123456', '9001765432', '9887654012',
        '9876540987', '9123678901', '9001230987', '9887654320', '9876012345'
    ],
    'City': [
        'Chennai', 'Coimbatore', 'Coimbatore', 'Tiruchirappalli', 'Salem',
        'Erode', 'Tirunelveli', 'Vellore', 'Thoothukudi', 'Dindigul',
        'Thanjavur', 'Kanchipuram', 'Chennai', 'Chennai', 'Chennai',
        'Coimbatore', 'Madurai', 'Tiruchirappalli', 'Salem', 'Erode',
        'Tirunelveli', 'Vellore', 'Thoothukudi', 'Dindigul', 'Thanjavur'
    ],
    'Pincode': [
        600010, 641021, 641021, 620043, 636054, 638065, 627076, 632087,
        628098, 624109, 613120, 631131, 600142, 600153, 600164,
        641175, 625186, 620197, 636208, 638219, 627230, 632241, 628252,
        624263, 613274
    ],
    'Gender': [
        'MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE',
        'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE',
        'MALE', 'FEMALE', 'MALE', 'FEMALE', 'MALE',
        'FEMALE', 'MALE', 'FEMALE', 'MALE', 'FEMALE',
        'MALE', 'FEMALE', 'MALE', 'FEMALE', 'FEMALE'
    ],
    'Age':[
        23,30,32,28,35,40,42,27,30,32,27,29,30,38,42,45,43,39,35,36,25,26,30,32,27
    ]

}

# Create the DataFrame
customers_df = pd.DataFrame(customers_data)

# Assign unique usernames
customers_df['userName'] = generate_customer_usernames(customers_df)

# Display the updated DataFrame
customers_df


,Name,password,Email,Address,Phone,City,Pincode,Gender,Age,userName
0,Harish Pillai,Harish@566#,harish.pillai@gmail.com,"No. 10, Lotus Street",9876543210,Chennai,600010,MALE,23,C_HARISHCHE001
1,Swetha Reddy,Swetha@633#,swetha.reddy@gmail.com,"No. 21, Maple Avenue",9123456780,Coimbatore,641021,FEMALE,30,C_SWETHACOI001
2,Swetha Shetty,Swetha@202#,manoj.shetty@gmail.com,"No. 32, Palm Grove",9988776655,Coimbatore,641021,FEMALE,32,C_SWETHACOI002
3,Deepa Suresh,Deepa@184#,deepa.suresh@gmail.com,"No. 43, Cedar Lane",9876501234,Tiruchirappalli,620043,FEMALE,28,C_DEEPATIR001
4,Rahul Nambiar,Rahul@466#,rahul.nambiar@gmail.com,"No. 54, Jasmine Road",9123409876,Salem,636054,MALE,35,C_RAHULSAL001
5,Lakshmi Prasad,Lakshmi@108#,lakshmi.prasad@gmail.com,"No. 65, Orchid Enclave",9001122334,Erode,638065,FEMALE,40,C_LAKSHMIERO001
6,Sandeep Varma,Sandeep@977#,sandeep.varma@gmail.com,"No. 76, Pine Residency",9090909090,Tirunelveli,627076,MALE,42,C_SANDEEPTIR001
7,Preeti Desai,Preeti@540#,preeti.desai@gmail.com,"No. 87, Willow Park",9887766554,Vellore,632087,FEMALE,27,C_PREETIVEL001
8,Ajay Kulkarni,Ajay@522#,ajay.kulkarni@gmail.com,"No. 98, Rose Drive",9876123450,Thoothukudi,628098,MALE,30,C_AJAYTHO001
9,Rina Fernandes,Rina@054#,rina.fernandes@gmail.com,"No. 109, Tulip Block",9123987654,Dindigul,624109,FEMALE,32,C_RINADIN001


## make sales_table

The `sales_df` DataFrame represents the sales transactions between car dealers and customers. Each row in this table records a single sale, capturing the following key details:

- **DealerUsername**: The unique username of the dealer who sold the car. This value is mapped from the `dealers_df` DataFrame.
- **customerUsername**: The unique username of the customer who purchased the car. This value is mapped from the `customers_df` DataFrame.
- **CarId**: The unique identifier for the car sold, generated and stored in the main `df` DataFrame.
- **brandId**: The unique brand identifier associated with the car, mapped using the `brand_id_map` dictionary and the car's make.

This table is essential for tracking which dealer sold which car to which customer, and for maintaining referential integrity between the dealers, customers, and cars in the database. The `sales_df` is also used to populate the `sales` table in the MySQL database, with appropriate foreign key relationships to ensure data consistency.

In [19]:
num_sales = 30

# For each sale, randomly pick a DealerUsername, then pick a CarId from df that matches that DealerUsername
dealer_usernames = dealers_df['userName'].dropna().sample(num_sales, replace=True, random_state=1).values
customer_usernames = customers_df['userName'].sample(num_sales, replace=True, random_state=2).values

car_ids = []
for dealer in dealer_usernames:
    # Get all Car_id for this dealer from df
    car_ids_for_dealer = df[df['DealerUsername'] == dealer]['Car_id'].values
    # If dealer has cars, pick one at random, else np.nan
    if len(car_ids_for_dealer) > 0:
        car_ids.append(np.random.choice(car_ids_for_dealer))
    else:
        car_ids.append(np.nan)

sales_df = pd.DataFrame({
    'DealerUsername': dealer_usernames,
    'customerUsername': customer_usernames,
    'CarId': car_ids,
})

# Map CarId to brandId using the df DataFrame
carid_to_brandid = dict(zip(df['Car_id'], df['Brand_ID']))
sales_df['brandId'] = sales_df['CarId'].map(carid_to_brandid)

# Add sales number column in the format S_001, S_002, ...
sales_df['sales_ID'] = ['S_{:03d}'.format(i+1) for i in range(len(sales_df))]

# Add random sales date between Jan 2025 and May 2025
date_range = pd.date_range(start='2025-01-01', end='2025-05-31', freq='D')
sales_dates = np.random.choice(date_range, size=num_sales)

# Generate random times between 9:00 and 18:00 (6:00 pm)
random_hours = np.random.randint(9, 18, size=num_sales)
random_minutes = np.random.randint(0, 60, size=num_sales)
random_seconds = np.random.randint(0, 60, size=num_sales)

# Combine date and time
sales_df['sales_date'] = [
    pd.Timestamp(date) + pd.Timedelta(hours=hour, minutes=minute, seconds=second)
    for date, hour, minute, second in zip(sales_dates, random_hours, random_minutes, random_seconds)
]

# Mark 2 customers who buy more than one car from 2 different dealers (optional, for analysis)
multi_buyers = sales_df.groupby('customerUsername').filter(lambda x: x['DealerUsername'].nunique() > 1)
special_customers = multi_buyers['customerUsername'].unique()[:2]  # pick first 2 such customers if available

sales_df

,DealerUsername,customerUsername,CarId,brandId,sales_ID,sales_date
0,D_SNEHAMAH001,C_AJAYTHO001,3399541C-60E4-4C7E-A106-1DEC04C42D30,MAH104,S_001,2025-02-25 14:16:57
1,D_POOJAMAR001,C_ANITACOI001,1435503D-2763-44EA-8EE9-C1C8EDC556BC,MAR109,S_002,2025-01-06 17:50:43
2,D_SENTHILTOY001,C_TANVICHE001,FD84C8B1-00C0-4005-B94E-CB123B4B26F5,TOY103,S_003,2025-04-11 16:30:10
3,D_SURESHBMW001,C_AJAYTHO001,75DE4879-925F-430B-B03C-B34C51A0A539,BMW102,S_004,2025-03-18 11:06:09
4,D_ARUNTAT002,C_SNEHATHO001,77B23DBF-D020-493A-AC55-8161AEA035D3,TAT106,S_005,2025-01-26 17:03:48
5,D_POOJAMAR001,C_MEENAKAN001,A1D4538F-C5B4-4F86-8C5F-9AC4B01C0D99,MAR109,S_006,2025-05-02 10:16:30
6,D_SNEHAMAH001,C_SUNITASAL001,1DB67510-AD35-4DDD-AA89-AC96EEC26DDC,MAH104,S_007,2025-05-30 17:14:53
7,D_REVATHIHON001,C_MEENAKAN001,55323C0C-C991-4E2B-B249-C5039352C717,HON107,S_008,2025-05-22 17:19:45
8,D_ARUNTAT001,C_AJAYTHO001,2E59616C-4807-4771-8B73-A4AC6F487F1E,TAT106,S_009,2025-01-23 09:09:59
9,D_GOPINATHMER001,C_PREETIVEL001,C183551C-DDE9-4758-B9B7-FDC17E964E12,MER105,S_010,2025-01-15 16:08:38


In [24]:
df[df['Car_id'] == '77B23DBF-D020-493A-AC55-8161AEA035D3']

,model,variant,Price_INR,engine_cc,Cylinders,fuel_capacity,Fuel_type,Body_type,mileage,Gears,...,warranty,Sun_visor,Ventilation_system,ABS,Airbags,color,rating,Brand_ID,DealerUsername,Car_id
373,Bolt,Xms Diesel,719661,1248,4,44,DIESEL,Hatchback,22.95,5,...,2,True,True,True,True,Brown,5,TAT106,D_ARUNTAT002,77B23DBF-D020-493A-AC55-8161AEA035D3


In [25]:
# Find customers who bought more than one car
multi_car_customers = sales_df['customerUsername'].value_counts()
multi_car_customers = multi_car_customers[multi_car_customers > 1]

# Print the usernames of such customers
print("Customers who bought more than one car:")
print(multi_car_customers)

Customers who bought more than one car:
customerUsername
C_MEENAKAN001      4
C_AJAYTHO001       3
C_PREETIVEL001     3
C_ANITACOI001      2
C_DEEPATIR001      2
C_PRIYATIR001      2
C_VIVEKTHA001      2
C_SANDEEPTIR001    2
Name: count, dtype: int64


In [26]:
# Find customers who have not bought any car
customers_with_sales = set(sales_df['customerUsername'])
customers_without_sales = customers_df[~customers_df['userName'].isin(customers_with_sales)]

print("Customers who have not bought any car:")
print(customers_without_sales[['Name', 'userName']])

Customers who have not bought any car:
              Name        userName
0    Harish Pillai  C_HARISHCHE001
9   Rina Fernandes    C_RINADIN001
12    Sahil Kapoor   C_SAHILCHE001
14      Yash Mehra    C_YASHCHE001
16     Rohit Verma   C_ROHITMAD001
23      Arjun Nair   C_ARJUNDIN001
24     Divya Gupta   C_DIVYATHA001


In [27]:
df.head()

,model,variant,Price_INR,engine_cc,Cylinders,fuel_capacity,Fuel_type,Body_type,mileage,Gears,...,warranty,Sun_visor,Ventilation_system,ABS,Airbags,color,rating,Brand_ID,DealerUsername,Car_id
0,Venue,1.0 Turbo Gdi Mt Sx Dual Tone,974000,998,4,45,PETROL,SUV,18.27,6,...,2,True,True,True,True,Silver,5,HYU101,D_PRIYAHYU001,E7DDE5E8-3C28-44C7-81E3-0655C9AC903D
1,X3,Xdrive20D Xline,5600000,1995,4,67,DIESEL,SUV,18.56,8,...,2,True,True,True,True,Green,5,BMW102,D_SURESHBMW001,75DE4879-925F-430B-B03C-B34C51A0A539
2,Fortuner,Trd Celebratory Edition,3420000,2755,4,80,DIESEL,SUV,12.90,6,...,3,True,True,True,True,Silver,5,TOY103,D_DIVYATOY001,FC2A430C-764B-4891-822D-5D7367CF90B6
3,Creta,1.6 Vtvt Sx,1232534,1591,4,55,PETROL,SUV,15.29,6,...,3,True,True,True,True,Maroon,5,HYU101,D_PRIYAHYU001,5B7DD4BD-5D1F-48E8-895B-4DBA501349CD
4,Nuvosport,N6 Amt,972359,1493,3,60,DIESEL,SUV,16.21,5,...,2,True,True,True,True,Yellow,4,MAH104,D_SNEHAMAH001,02CC8F43-A3DA-48B8-AE4C-DE9AAFA8350F


In [28]:
import os

# Create the data folder if it doesn't exist
os.makedirs('data', exist_ok=True)

# Save all relevant DataFrames to Excel files in the data folder
with pd.ExcelWriter('data/cars.xlsx') as writer:
    df.to_excel(writer, index=False)
with pd.ExcelWriter('data/dealers.xlsx') as writer:
    dealers_df.to_excel(writer, index=False)
with pd.ExcelWriter('data/customers.xlsx') as writer:
    customers_df.to_excel(writer, index=False)
with pd.ExcelWriter('data/sales.xlsx') as writer:
    sales_df.to_excel(writer, index=False)
with pd.ExcelWriter('data/car_brands.xlsx') as writer:
    car_brands.to_excel(writer, index=False)

In [29]:
dealers_df

,Name,password,Email,Phone,Address,City,pincode,Age,gender,description,userName,Brand_ID
0,Arun Kumar,Arun@506$,arun.kumar@gmail.com,9876543210,"No. 12, Anna Nagar, Chennai",Chennai,600001,35,MALE,Experienced Tata dealer in Chennai.\nCommitted...,D_ARUNTAT001,TAT106
1,Priya Sharma,Priya@028$,priya.sharma@gmail.com,9123456780,"No. 45, Gandhi Road, Coimbatore",Coimbatore,641001,40,FEMALE,Leading Hyundai dealer in Coimbatore.\nKnown f...,D_PRIYAHYU001,HYU101
2,Ramesh Babu,Ramesh@541$,ramesh.babu@gmail.com,9988776655,"No. 78, MG Street, Madurai",Madurai,625001,28,MALE,Maruti Suzuki specialist in Madurai.\nFriendly...,D_RAMESHMAR001,MAR109
3,Divya Menon,Divya@564$,divya.menon@gmail.com,9876501234,"No. 23, Park Avenue, Tiruchirappalli",Tiruchirappalli,620001,45,FEMALE,Toyota dealership in Tiruchirappalli.\nFocus o...,D_DIVYATOY001,TOY103
4,Karthik Raja,Karthik@097$,karthik.raja@gmail.com,9123409876,"No. 56, Raja Street, Salem",Salem,636001,32,MALE,Ford dealer serving Salem region.\nCustomer-fi...,D_KARTHIKFOR001,FOR108
5,Sneha Iyer,Sneha@010$,sneha.iyer@gmail.com,9988123456,"No. 89, Lake View, Erode",Erode,638001,38,FEMALE,Mahindra expert in Erode.\nDedicated to after-...,D_SNEHAMAH001,MAH104
6,Vijay Anand,Vijay@519$,vijay.anand@gmail.com,9876123450,"No. 34, Temple Road, Tirunelveli",Tirunelveli,627001,29,MALE,Honda dealership in Tirunelveli.\nPersonalized...,D_VIJAYHON001,HON107
7,Meera Nair,Meera@826$,meera.nair@gmail.com,9123987654,"No. 67, Green Park, Vellore",Vellore,632001,50,FEMALE,Mercedes-Benz dealer in Vellore.\nLuxury car s...,D_MEERAMER001,MER105
8,Suresh Reddy,Suresh@866$,suresh.reddy@gmail.com,9988345678,"No. 90, Sunrise Avenue, Thoothukudi",Thoothukudi,628001,33,MALE,BMW dealership in Thoothukudi.\nLatest models ...,D_SURESHBMW001,BMW102
9,Arun Sharma,Arun@955$,arun.gupta@gmail.com,9876234567,"No. 11, Hilltop Road, Dindigul",Dindigul,624001,41,FEMALE,Tata dealer in Dindigul.\nQuick delivery and s...,D_ARUNTAT002,TAT106
